In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import joblib
import serial

app = Flask(__name__)
model = joblib.load('Soul.pkl')
value=[]  
def serialget():
    ser = serial.Serial()
    ser.port = 'COM5'
    ser.baudrate = 9600
    ser.bytesize = 8
    ser.parity = serial.PARITY_NONE
    ser.stopbits = serial.STOPBITS_ONE
    ser.open()
    v=b'A'
    ser.write(v)
    while True:
        for line in ser.read():
            if chr(line) != '$':
                value.append(chr(line))
            else:
                print("end")
                ser.close()
                return value
def serialset(v):
    ser = serial.Serial()
    ser.port = 'COM5'
    ser.baudrate = 9600
    ser.bytesize = 8
    ser.parity = serial.PARITY_NONE
    ser.stopbits = serial.STOPBITS_ONE
    ser.open()
    ser.write(v)
    
                                         
@app.route('/')
def home():
    return render_template('index.html')


@app.route('/request1')
def request1():
    str1=''
    print('smb')
    val=[]
    va=serialget()
    print (va)
    for v in va:
        if(v=='*'):
            continue
        else:  
            if(v!='#'): 
                str1+=v
            else:
                print(str1)
                val.append(float(str1))
                str1=""   
    ldr=val[2]
    if(ldr==0):
        ldr=1
    elif(ldr==1):
        ldr=0
    return render_template('index.html',val1=val[0],val2=val[1],val3=ldr)   
    
    
@app.route('/predict',methods=['POST'])

def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    print(final_features)
    prediction = model.predict(final_features)

    output = prediction[0]
    print(output)
    if(output>6982):
        v=b'B'
        serialset(v)
        view="Power High"
    elif(output<6982):
        v=b'C'
        serialset(v)
        view="Power Low"
    return render_template('index.html', prediction_text='SOLAR POWER IS {}'.format(output),value=view)

if __name__ == "__main__":
    app.run(host="localhost", port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2022 21:14:15] "GET / HTTP/1.1" 200 -
